<img src="img/nomiente.png">

# Proyecto de aula - Reconocimiento de Patrones
## Jhon Jaime Gil Sepulveda - Nelson Javier Posada Flórez

Para este proyecto se requiere una base de datos de 100 comentarios de usuarios en el ambito de hotelería. Estos datos deben ser balanceados por lo que se deberán sacar 50 comentarios positivos y 50 comentarios negativos.

### Art Hotel Boutique Medellín
Hemos elegido este hotel por la cantidad de comentarios disponibles en la página de Booking.com
Puedes encontrar el link de este aquí:

[Art Hotel Boutique in Booking.com](https://www.booking.com/hotel/co/art.es.html?label=gen173nr-1FCAEoggJCAlhYSDNYBGgyiAEBmAEKwgEKd2luZG93cyAxMMgBDNgBAegBAfgBC5ICAXmoAgM;sid&#tab-main)

### Reporte de artículos
El reporte se puede encontrar dentro de la carpeta docs/reporte.pdf

### Proyecto

#### 1. Extracción de comentarios
Dado que la recolección de comentarios parecía una tarea sencilla, quisimos automatizar este proceso por lo que se utilizó una técnica para extraer los reviews del hotel llamada [WebScrapping](https://en.wikipedia.org/wiki/Web_scraping)
Haciendo uso de la librería Selenium y BeautifulSoup, se realizó el proceso automatico de extracción de los documentos y el código puede ser encontrado dentro de la carpeta "utils/BookingScrapper.py"
Posteriormente a esto, los comentarios se corrigieron manualmente, estos quedaron guardados en el archivo reviews.txt.

#### 2. Caracterización de texto
Se hizo el uso de la API [Corpus](http://www.corpus.unam.mx/servicio-freeling/) para obtener el etiquetado morfosintáctico de los comentarios y posteriormente a esto, se utilizó el recurso léxico MLSenticon para obtener los pesos que representaban los datos (Se tuvo en cuenta la polaridad del texto, por lo que a esto se le diseñó un algorimo sencillo también).

#### 3. Modelos clasificadores
Se utilizó la librería Scikit Learn para obtener las medidas de desempeño de los datos recibidos del anterior punto, se utilizó la métodología de validación Boostraping y se presentaron los datos en una tabla utilizando la libería Pandas.

#### 4. TF-IDF y CountVectorizer
Nuevamente, haciendo uso de la librería Scikit Learn utilizamos los recursos de TF-IDK y CountVectorizer. Posteriormente a esto, se realizó nuevamente el punto 3 con los valores nuevos y se presentaron los datos en una tabla utilizando la libería Pandas.

##### El proyecto también puede ser ejecutado ejecutando "python Proyecto.py"

[Proyecto en Github](https://github.com/RanKey1496/BookingScrapper)

In [1]:
import requests
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier as RF

In [2]:
"""
Se leen los comentarios y se devuelven dos listas con los valores
de X y Y
"""
def get_database():
    db = open('reviews.txt', 'r')
    db_data = []
    db_target = []
    for line in db:
        db_data.append(line.split('\t')[0])
        db_target.append(line.split('\t')[1][:-1])
    db.close()
    return db_data, db_target

"""
Retorna si una palabra existe dentro una lista
"""
def exists(array, word):
    for i in array:
        if (i[0] == word):
            return i
    return None

"""
´Mira si una palabra es una negación o no
"""
def is_negative(text):
    low = text.lower()
    negatives = ['no', 'pero', 'aunque', 'but', 'ni']
    if low in negatives:
        return True
    return False

"""
Copia cada comentario en un archivo dentro de la carpeta "reviews"
"""
def write_comments_in_separeted_files():
    file = open('reviews.txt', 'r').readlines()
    for i, line in enumerate(file):
        newFile = open('reviews/review'+str(i)+'.txt', 'w')
        newFile.write(file[i].split('\t')[0])
        newFile.close()
        
"""
Retorna todos los nombres de archivos en la carpeta "reviews"
"""        
def get_file_names():
    for root, dirs, files in os.walk('./reviews'):
        return files

In [3]:
"""
Obtiene la lista de valores de la BD de Senticon
"""
def get_senticon():
    listSenticon = []
    mlSenticon = open('MLSenticon.txt', 'r')
    for i in mlSenticon:
        word = i.split('\t')
        listSenticon.append([word[0], word[1].replace('\n', '')])
    mlSenticon.close()
    return listSenticon

"""
Hace un POST request a la API Corpues con un archivo que
contiene un comentario y esta nos devuelve el objeto JSON
de la respuesta
"""
def obtain_freeling(fileName):
    files = {'file': open(fileName, 'r')}
    params = {'outf': 'tagged', 'format': 'json'}
    url = 'http://www.corpus.unam.mx/servicio-freeling/analyze.php'
    req = requests.post(url, files=files, params=params)
    return req.json()

"""
Recibe un objeto respuesta de la API Corpus y la BD de Senticon
Si la palabra es una negación invierte los pesos del Senticon,
posteriormente si la palabra es un adjetivo, si el peso de la palabra
por la polaridad es mayor a 0, sumará el valor del peso a la
variable de los positivos, en caso contrario a los negativos
Si la palabra es un punto y la polaridad es negativa, se invierte la
polaridad nuevamente y al finalizar de analizar cada objeto obtenido
de la API Corpus retornamos los valores
"""
def classification(res, senticon):
    x1, x2 = 0, 0
    polarity = 1
    for r in res:
        for word in r:
            if (is_negative(word['token'])):
                polarity = -1
            if (word['tag'][0] == 'A'):
                e = exists(senticon, word['lemma'])
                if (e):
                    if (float(e[1])*polarity > 0):
                        x1 += float(e[1])
                    else:
                        x2 += float(e[1])
            if (polarity == -1 and word['token'] == '.'):
                polarity = 1
    return x1, x2

In [4]:
"""
Retorna la sensibilidad y la especificidad de los datos
obtenidos contra los datos de prueba
"""
def error_measures(Ypredict, Yreal):
    CM = confusion_matrix(Yreal, Ypredict)
    
    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    
    sens = TP/(TP+FN)
    spec = TN/(TN+FP)
    return sens, spec

"""
Imprime los datos en una tabla haciendo uso de la libreria pandas
"""
def table_result(data, index, model):
    print('\n', model)
    df = pd.DataFrame(data, index=index)
    print(df)

### Modelos clasificadores
Todas las funciones retornan los valores de exactitud, sensibilidad y especificidad.
Para los modelos clasificadores de Logistic Regression y K-Nearest Neighbors se utilizó la función del modelo .score para obtener la exactitud de los datos, mientras que para el modelo Random Forest se hizo uso del modulo acurracy_score de Scikit Learn

In [5]:
"""
Haciendo uso del modelo Logistic Regression
"""
def lr_classification(data, y):
    model = LR()
    acc = []
    sens = []
    spec = []
    
    for i in range(100):
        Xtrain, Xtest, Ytrain, Ytest = train_test_split(data, y)
        model.fit(Xtrain, Ytrain)
        y_pred = model.predict(Xtest)
        sen, spc = error_measures(y_pred, Ytest)
        sens.append(sen)
        spec.append(spc)
        acc.append(model.score(Xtest, Ytest))
    return acc, sens, spec

"""
Haciendo uso del modelo K-Nearest Neighbors
"""
def knn_classification(data, y, k):
    model = KNN(n_neighbors=k)
    acc = []
    sens = []
    spec = []
    
    for i in range(100):
        Xtrain, Xtest, Ytrain, Ytest = train_test_split(data, y)
        model.fit(Xtrain, Ytrain)
        y_pred = model.predict(Xtest)
        sen, spc = error_measures(y_pred, Ytest)
        sens.append(sen)
        spec.append(spc)
        acc.append(model.score(Xtest, Ytest))
    return acc, sens, spec

"""
Haciendo uso del modelo Random Forest
"""
def rf_classification(data, y, estimators):
    model = RF(n_estimators=estimators)
    acc = []
    sens = []
    spec = []
    
    for i in range(100):
        Xtrain, Xtest, Ytrain, Ytest = train_test_split(data, y)
        model.fit(Xtrain, Ytrain)
        y_pred = model.predict(Xtest)
        sen, spc = error_measures(y_pred, Ytest)
        sens.append(sen)
        spec.append(spc)
        acc.append(accuracy_score(Ytest, y_pred))
    return acc, sens, spec

### Procesamiento
Tomando los datos obtenidos de los comentarios, se realiza el procesamiento de los datos de acuerdo a los párametros dados por el profesor y luego imprimiendolos en una tabla usando la librería pandas

In [27]:
def lr_results(data, target):
    lr = lr_classification(data, target)
    data = {'Accurancy': '{} +/- {}'.format(round(np.mean(lr[0]),4), round(np.std(lr[0]),2)),
            'Sensibility': '{} +/- {}'.format(round(np.mean(lr[1]),4), round(np.std(lr[1]),2)),
            'Specificity': '{} +/- {}'.format(round(np.mean(lr[1]),4), round(np.std(lr[2]),2))}
    table_result(data, [1], 'Logistic Regression Classification')

def knn_results(data, target):
    ks = [1,3,5,7,9,15,25]
    acc = []
    sens = []
    spec = []
    for i in ks:
        knn = knn_classification(data, target, i)
        acc.append('{} +/- {}'.format(round(np.mean(knn[0]),4), round(np.std(knn[0]),2)))
        sens.append('{} +/- {}'.format(round(np.mean(knn[1]),4), round(np.std(knn[1]),2)))
        spec.append('{} +/- {}'.format(round(np.mean(knn[2]),4), round(np.std(knn[2]),2)))
    data = {'Accurancy': acc, 'Sensibility': sens, 'Specificity': spec}
    df = pd.DataFrame(data, index=ks)
    table_result(data, ks, 'K-Nearest Neighbors Classification')
    
def rf_results(data, target):
    n_estimators = [10,20,30,40,50]
    acc = []
    sens = []
    spec = []
    for i in n_estimators:
        rf = rf_classification(data, target, i)       
        acc.append('{} +/- {}'.format(round(np.mean(rf[0]),4), round(np.std(rf[0]),2)))
        sens.append('{} +/- {}'.format(round(np.mean(rf[1]),4), round(np.std(rf[1]),2)))
        spec.append('{} +/- {}'.format(round(np.mean(rf[2]),4), round(np.std(rf[2]),2)))
    data = {'Accurancy': acc, 'Sensibility': sens, 'Specificity': spec}
    table_result(data, n_estimators, 'Random Forest Classification')

# Ejecución
#### En este paso pasará la mágia, así que pongase cómodo y unas gafas por no sabemos que sucederá.


<img src="img/giphy.gif">

In [6]:
"""
Obtener los datos de las bases de datos, tanto comentarios como el MLSenticon.
Escribimos los comentarios en archivos separados para facilitar el trabajo con
la API Corpus y todos los nombres que hay dentro de la carpeta con los archivos
"""
data, target = get_database()
senticon = get_senticon()
write_comments_in_separeted_files()
files = get_file_names()

In [7]:
"""
Por cada comentario (archivo), haremos la petición a la API Corpus, luego
se hará la clasificación con respecto al MLSenticon y sus pesos
"""
x = []
for i in files:
    review = './reviews/' + i
    req = obtain_freeling(review)
    result = classification(req, senticon)
    x.append([result[0], result[1]])

In [13]:
"""
Los valores obtenidos por la API Corpus y MLSenticon
"""
print(x)

[[0, 0], [0.75, 0], [0, 0], [0, 0], [3.364, 0], [0.292, 0], [0, 0], [0, 0], [2.416, 0], [0, 0.25], [0.833, 0], [0, 0.612], [2.136, 0], [0.75, 0], [0.5840000000000001, 0], [3.575, 0], [0.833, 0], [1.5, 0], [0, 0.347], [1.2189999999999999, 0], [0, 0], [0, 0], [0, 0], [1.375, 0], [1.3479999999999999, 0], [0.406, 1.666], [2.724, 0], [0.75, -0.575], [0.833, 0], [0, 0], [2.271, 0], [-1.908, 0], [0, 0], [0, 0], [0.675, -0.575], [1.125, 0], [-0.538, 0], [0.75, 0], [0, 0], [1.458, 2.416], [0, 0.16700000000000004], [0.792, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0.17500000000000004, -0.5], [1.174, 0], [0, 0], [0, 0], [0, 0], [0, 0], [1.875, 0], [0, 0], [0, 0], [0, 0], [0.833, 0], [0, 0], [0, 0], [0, 0], [0.625, 0], [0, -0.575], [0, 0], [0, 0], [0, 0], [2.661, 0], [0.25, 0], [0, 0], [0, 0], [2.354, 0], [0, 0], [0, 0], [0, 1.4789999999999999], [0, 0], [0, 0], [0, -0.575], [2.354, 0], [0, -0.25], [0.458, 0], [0, -0.281], [0.75, 0], [0, 0], [0.333, 2.4989999999999997], [0, 0], [

In [28]:
"""
Evaluamos estos datos con el modelo Logistic Regression
"""
lr_results(x, target)

"""
Evaluamos estos datos con el modelo K-Nearest Neighbors
"""
knn_results(x, target)

"""
Evaluamos estos datos con el modelo Random Forest
"""
rf_results(x, target)


 Logistic Regression Classification
         Accurancy      Sensibility      Specificity
1  0.4296 +/- 0.07  0.4681 +/- 0.33  0.4681 +/- 0.35

 K-Nearest Neighbors Classification
          Accurancy      Sensibility      Specificity
1   0.4682 +/- 0.09   0.421 +/- 0.22    0.517 +/- 0.3
3   0.4857 +/- 0.09  0.4634 +/- 0.24   0.5197 +/- 0.3
5   0.5043 +/- 0.09  0.4447 +/- 0.25  0.5848 +/- 0.31
7   0.5043 +/- 0.08  0.4452 +/- 0.23  0.5787 +/- 0.31
9    0.4907 +/- 0.1  0.4286 +/- 0.25   0.5758 +/- 0.3
15  0.5139 +/- 0.09  0.4492 +/- 0.21  0.6051 +/- 0.29
25   0.4889 +/- 0.1  0.4503 +/- 0.24    0.57 +/- 0.29

 Random Forest Classification
          Accurancy      Sensibility      Specificity
10  0.4707 +/- 0.08   0.292 +/- 0.17   0.6698 +/- 0.2
20  0.4729 +/- 0.09  0.2845 +/- 0.16  0.6747 +/- 0.21
30  0.4811 +/- 0.08  0.2864 +/- 0.13   0.6998 +/- 0.2
40  0.4946 +/- 0.08  0.2687 +/- 0.13  0.7282 +/- 0.16
50  0.4804 +/- 0.08  0.2855 +/- 0.14  0.6917 +/- 0.21


In [29]:
"""
Obtenemos los datos nuevamente y con la representación de los textos
haciendo uso de TF-IDF Vectorizer de Scikit Learn realizamos una 
nueva evaluación con los 3 modelos de clasificación
"""
data, target = get_database()
vectorTF = TfidfVectorizer()
vectorTF.fit(data)
bow = vectorTF.transform(data)
lr_results(bow, target)
knn_results(bow, target)
rf_results(bow, target)


 Logistic Regression Classification
         Accurancy     Sensibility      Specificity
1  0.8946 +/- 0.06  0.8636 +/- 0.1  0.8636 +/- 0.08

 K-Nearest Neighbors Classification
          Accurancy      Sensibility      Specificity
1   0.8639 +/- 0.06  0.9144 +/- 0.07  0.8144 +/- 0.09
3   0.8879 +/- 0.05  0.9258 +/- 0.06  0.8503 +/- 0.08
5   0.8861 +/- 0.05  0.9244 +/- 0.07  0.8551 +/- 0.08
7   0.8825 +/- 0.06  0.9133 +/- 0.08  0.8565 +/- 0.08
9    0.875 +/- 0.05  0.8892 +/- 0.08  0.8681 +/- 0.08
15  0.8668 +/- 0.05  0.8873 +/- 0.08  0.8602 +/- 0.09
25  0.8564 +/- 0.06  0.8553 +/- 0.08  0.8683 +/- 0.09

 Random Forest Classification
          Accurancy      Sensibility      Specificity
10  0.7746 +/- 0.08  0.8107 +/- 0.12  0.7517 +/- 0.14
20  0.7882 +/- 0.08  0.8429 +/- 0.11  0.7497 +/- 0.14
30  0.8011 +/- 0.07    0.861 +/- 0.1  0.7531 +/- 0.12
40  0.7943 +/- 0.07  0.8818 +/- 0.09  0.7195 +/- 0.12
50  0.7975 +/- 0.07   0.8627 +/- 0.1   0.746 +/- 0.12


In [ ]:
"""
Ahora nuevamente, obtenemos los datos y con la matriz de terminos de
documentos CountVectorizer de Scikit Learn realizamos una nueva evaluación
con los 3 modelos de clasificación
"""
data, target = get_database()
vectorCount = CountVectorizer(ngram_range=(1,2))
vectorCount.fit(data)
bow = vectorCount.transform(data)
lr_results(bow, target)
knn_results(bow, target)
rf_results(bow, target)


 Logistic Regression Classification
         Accurancy      Sensibility      Specificity
1  0.8639 +/- 0.07  0.9168 +/- 0.07  0.9168 +/- 0.11

 K-Nearest Neighbors Classification
          Accurancy      Sensibility      Specificity
1   0.6393 +/- 0.08  0.8487 +/- 0.19  0.4231 +/- 0.19
3   0.5786 +/- 0.09  0.9784 +/- 0.04  0.1869 +/- 0.13
5   0.5593 +/- 0.09  0.9992 +/- 0.01  0.1213 +/- 0.09
7   0.5518 +/- 0.09  0.9994 +/- 0.01  0.0914 +/- 0.08
9   0.5361 +/- 0.09      1.0 +/- 0.0  0.0659 +/- 0.05
15  0.5229 +/- 0.08      1.0 +/- 0.0   0.058 +/- 0.05
25  0.5193 +/- 0.09      1.0 +/- 0.0    0.04 +/- 0.05


# Esto es todo señores

<img src="img/miente.png">

##### PD: Profe no nos rebaje por esta estupidez :'v